In [ ]:
import sys
import os
import torch
from torch import nn
sys.path.append(os.path.abspath('../python'))
sys.path.append('../site-packages/nvidia-ml-py')

In [ ]:
from moire_screen_gan import Net
m = Net()

In [ ]:
import re
from functools import reduce
getRoot = lambda w, r: tuple(filter(lambda s: s.startswith(r), w.keys()))
replaces = lambda w, r, p, s: list((t, t.replace(p, s)) for t in getRoot(w, r + p))
inserts = lambda w, r, p, s: list((t, t.replace(r + p, r + s + p)) for t in getRoot(w, r + p))
getMatch = lambda o: o.group(1) if len(o.groups()) else o.group(0)
def find(names, r):
    r = re.compile(r)
    return set(map(getMatch, filter(None, map(r.match, names))))
findRs = lambda w, rs: reduce(set.union, (find(tuple(w.keys()), r) for r in rs))
def changeName(w, old, new):
    if new:
        w[new] = w[old]
    del w[old]
def changeNames(w, names):
    for old, new in names:
        changeName(w, old, new)
pf = lambda f, w, r: lambda p, s: [] if p is None else f(w, r, p, s)
getNames = lambda w: lambda rst: reduce(lambda a, r: a + pf(replaces, w, r)(rst[1], rst[2]) + pf(inserts, w, r)(rst[3], rst[4]), findRs(w, rst[0]), [])
cc = lambda w: lambda rsts: tuple(changeNames(w, getNames(w)(rst)) for rst in rsts)
removeRoot = lambda w, r: tuple(changeName(w, old, None) for old in getRoot(w, r))

In [ ]:
modelPath = '../model/demoire/moire_screen_gan.pth'
weights = torch.load(modelPath, map_location='cpu')

getW = getNames(weights)
ccW = cc(weights)

In [ ]:
#modelPath = 'moire_t.pth'
torch.save(weights, modelPath, pickle_protocol=4)

In [ ]:
weights = torch.load('moire_t.pth', map_location='cpu')

In [ ]:
list(weights.keys())

In [ ]:
w1 = m.state_dict()
list(w1.keys())

In [ ]:
fm1 = lambda i, s0, s1, s2: (s0.format(i + 1), s1.format(i + 1), s2.format(i))
reT = lambda t: ((t[0],), t[1], t[2], None, None)
rsts = [
    ((r'branch.\.convt_F..\.',), 'ca1.', '0.ca.', None, None),
    ((r'branch.\.convt_F..\.',), 'ca2.', '1.ca.', None, None),
    ((r'branch.\.convt_F..\.',), 'relu1.', '0.relu.', None, None),
    ((r'branch.\.convt_F..\.',), 'relu2.', '1.relu.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv1.', '0.conv1.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv2.', '0.conv2.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv3.', '1.conv1.', None, None),
    ((r'branch.\.convt_F..\.',), 'conv4.', '1.conv2.', None, None),
    ((r'branch.\.convt_F..\.[01]\.',), None, None, '', '0.'),
    ((r'branch.\.u.\.',), 'conv.', '0.', None, None),
    ((r'branch.\.u.\.',), 'prelu.', '2.', None, None),
    ((r'branch.\.non_local\.non_local\.[^.]+\.',), '0.', '', None, None)
] + [
    reT(fm1(i, r'(branch.\.)convt_F1{}\.', 'convt_F1{}.', 'convt_F.{}.')) for i in range(8)
] + [
    reT(fm1(i, r'(branch.\.)s_conv{}\.', 's_conv{}.', 's_conv.{}.')) for i in range(8)
] + [
    reT(fm1(i, r'(branch.\.)u{}\.', 'u{}.', 'u.{}.')) for i in range(8)
] + [
    reT(fm1(i, r'()branch{}\.', 'branch{}.', 'branches.{}.')) for i in range(5)
] + [
    reT(fm1(i, r'()scale{}\.', 'scale{}.', 'scales.{}.')) for i in range(5)
] + [
    reT(fm1(i, r'()down2_{}\.', 'down2_{}.', '_down2.{}.')) for i in range(4)
]

In [ ]:
removeRoot(weights, 'branch6')
removeRoot(weights, 'scale6')
removeRoot(weights, 'down2_5')
ccW(rsts)
m.load_state_dict(weights)

In [ ]:
torch.save(m.state_dict(), modelPath)